In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
pd.__version__

'1.4.2'

In [4]:
cd ..

/workspaces/MLops-zoomcamp


In [5]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [6]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [7]:
len(df.columns)

19

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
df['duration'].std()

42.594351241920904

In [10]:
filtered_df = df[(df['duration'] >=1) & (df['duration'] <=60)]
fraction_left = len(filtered_df) / len(df)
fraction_left

0.9812202822125979

In [11]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [12]:
df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [13]:
dv = DictVectorizer(sparse=True)

In [15]:
# len(dv.feature_names_)

In [16]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649143388169879

In [17]:
# read in the validation set
yellow_feb_23 = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

# create a new column 'duration' in minutes
yellow_feb_23['duration'] = yellow_feb_23.tpep_dropoff_datetime - yellow_feb_23.tpep_pickup_datetime
# convert duration to seconds
yellow_feb_23.duration = yellow_feb_23.duration.apply(lambda td: td.total_seconds()/60)

# define categorical and numerical columns
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# convert categorical columns to strings
yellow_feb_23[categorical] = yellow_feb_23[categorical].astype(str)

# remove outliers by clipping duration between 1 and 60 mins
yellow_feb_23 = yellow_feb_23[(yellow_feb_23.duration >= 1) & (yellow_feb_23.duration <= 60)]

In [18]:
# Instantiate a dictionary vectorizer
dv = DictVectorizer()
train_dicts = df[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

# turn the categorical columns into a list of dictionaries
val_dicts = yellow_feb_23[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# define target variable and assign to y_val
target = 'duration'
y_train = df[target].values
y_val = yellow_feb_23[target].values

# instantiate & fit our model to the TRAINING set
lr = LinearRegression()
lr.fit(X_train,y_train)


LinearRegression()

In [19]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.649343165600712